In [2]:
import pandas as pd
from icecream import ic
import spacy
import progressbar
from spacy.matcher import Matcher, DependencyMatcher
# !python -m spacy download en_core_web_sm
nlp=spacy.load("en_core_web_sm")


In [3]:
data=pd.read_csv('dataframe_true_final.csv')
text=data['txt']
data=data.drop(columns=['Unnamed: 0','txt','id'])
text=text.str.replace("'","",regex=False)
text=text.str.replace("]","",regex=False)
text=text.str.replace("[","",regex=False)
text=text.str.replace(",","",regex=False)
text=text.str.replace("A ","a ",regex=False)
text=text.str.replace("And","and",regex=False)

In [4]:
newdata=pd.DataFrame(index=text.index, columns=['PERSON','JOB','NORP','GPE','ORG'])

matcher_nom=DependencyMatcher(nlp.vocab)
pattern_nom=[[{"RIGHT_ID":"nom","RIGHT_ATTRS":{"DEP":"nsubj"}},{"LEFT_ID":"nom","REL_OP":">","RIGHT_ID":"reste","RIGHT_ATTRS":{}}]]
matcher_nom.add("nom",pattern_nom)

matcher_job = Matcher(nlp.vocab)
pattern_job = [[{"DEP": "attr"}]]
matcher_job.add("JOB", pattern_job)


In [5]:
wrong=set()
for i in progressbar.progressbar(range (100)):
  doc=nlp(text[i])
  A=matcher_nom(doc)
  B=matcher_job(doc)
  if A!=[]:   
    newdata.loc[i]['PERSON']=doc[A[0][1][1]:A[0][1][0]+1].text
  else: 
    wrong.add(i)
  if B!=[]:   
    newdata.loc[i]['JOB']=doc[B[0][1]:B[0][2]].text
  else: 
    wrong.add(i)


  for col in ['NORP','GPE','ORG']:
    for ent in doc.ents:
      if ent.label_==col and newdata.iloc[i][col]!=newdata.iloc[i][col]:
        newdata.loc[i][col]=ent.text



for df in [newdata, text]:
  df.drop(wrong).reset_index().drop(columns=['index'])


100% (100 of 100) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [29]:
def essai(i): #tester le nouveau dataset
  ic(text[i])
  doc=nlp(text[i])
  for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)




In [64]:
essai(130)

ic| text[i]: 'Heiko Engelkes was a German journalist'
Heiko PROPN compound Engelkes
Engelkes NOUN nsubj was
was AUX ROOT was
a DET det journalist
German ADJ amod journalist
journalist NOUN attr was


In [10]:
# pd.concat([text,newdata],axis=1).to_csv('New_dataset.csv')

In [76]:
matcher_job=DependencyMatcher(nlp.vocab)
pattern_job=[[{"RIGHT_ID":"nom","RIGHT_ATTRS":{"DEP":"attr"}},{"LEFT_ID":"nom","REL_OP":">>","RIGHT_ID":"reste","RIGHT_ATTRS":{"DEP":{"IN": ["amod", "compound"]}}}]]
matcher_job.add("nom",pattern_job)

matcher_attr = Matcher(nlp.vocab)
pattern_attr = [[{"DEP": "attr"}]]
matcher_attr.add("JOB", pattern_attr)

# matcher_job=DependencyMatcher(nlp.vocab)
# pattern_job=[[{"RIGHT_ID":"gauche","RIGHT_ATTRS":{}},{"LEFT_ID":"gauche","REL_OP":"<<","RIGHT_ID":"job","RIGHT_ATTRS":{"DEP":"attr"}},{"LEFT_ID":"job","REL_OP":">","RIGHT_ID":"reste","RIGHT_ATTRS":{}}]]
# matcher_job.add("nom",pattern_job)



In [87]:
doc=nlp(text[165])
ic(doc)
A=matcher_job(doc)
B=matcher_attr(doc)
for x in A:
    ic(x[1])
    y=sorted(x[1])
    ic(y)
    ic(doc[y[0]:y[1]])
ic(doc[B[0][1]:B[0][2]])

ic| doc: Hans Behrendt was a German Jewish actor screenwriter and film director
ic| x[1]: [10, 4]
ic| y: [4, 10]
ic| doc[y[0]:y[1]]: German Jewish actor screenwriter and film
ic| x[1]: [10, 5]
ic| y: [5, 10]
ic| doc[y[0]:y[1]]: Jewish actor screenwriter and film
ic| x[1]: [10, 6]
ic| y: [6, 10]
ic| doc[y[0]:y[1]]: actor screenwriter and film
ic| doc[B[0][1]:B[0][2]]: director


director

In [167]:

# Z=doc[i[1][1]:i[1][0]+1]
# for j in range(len(Z)):
#     ic(j)
#     ic(Z[j:j+1])
#     if Z[j:j+1].ents!=[]:
#         if Z[j:j+1].ents[0].label_=='NORP':
#             break

# ic(j)

ic| j: 0
ic| Z[j:j+1]: German
ic| j: 0


0